In [ ]:
%%script false --no-raise-error
#Copy this code and run it in the terminal everytime you create a new directory or file
#Gives permission for everyone to work with the files created
#Run at the end of a jupyter script, the output is too messy
!cd /cephyr/NOBACKUP/groups/bbt045_2025/groups/group_Bjork 
!chmod -R 777 . 

# Create transcript library 
1. Download human genome from ensembl: https://ftp.ensembl.org/pub/release-113/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
2. Create transcript fasta with bedtools getfasta.

In [ ]:
%%bash
cd /cephyr/NOBACKUP/groups/bbt045_2025/groups/group_Bjork
TRANSCRIPT_FILE='data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.cdna.all.fa'
GTF_ANNOTATION='data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.113.gtf'

# 1. 
# -nc makes sure that if the file is already there, it won't be overwritten.
# -nc can be changed to -N, if it is of interest to import newer versions
# -P will save it to the directory specified, in this case data/cDNA_library
# wget -nc https://ftp.ensembl.org/pub/release-113/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz -P data/homo_sapiens_ensembl
# gzip -d data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
# rm data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz

# 2.
# bedtools getfasta \
# -fi $GENOME_FILE \
# -bed $GTF_ANNOTATION \
# -fo data/homo_sapiens_ensembl/transcriptome_assembly.fa
# remove unzipped genome to save server space
# rm data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.dna.primary_assembly.fa

# Build Bowtie index
This takes around 30 minutes to run on one kernel, so don't run it if you don't have to. For the bedtools index: started ~18:12 on two kernels, ended before 18:24

In [ ]:
%%script false --no-raise-error
%%bash
# does not run in jupyter, run in terminal
# double check what features there are in the cDNA file
grep ">" Homo_sapiens.GRCh38.genome_like.fa | cut -d " " -f3 | cut -d ":" -f1 | sort | uniq -c
# output: 187466 chromosome 19667 scaffold

awk 'BEGIN {OFS=" "} 
/^>/ { 
    if (match($0, /chromosome:GRCh38:([0-9XYM]+)/, chr)) { 
        match($0, /chromosome:GRCh38:[0-9XYM]+:[0-9]+:([0-9]+)/, end);
        print ">" chr[1], "dna_sm:chromosome chromosome:GRCh38:" chr[1] ":1:" end[1] ":1 REF"; 
    } 
    else if (match($0, /scaffold:GRCh38:([^:]+)/, scaffold)) { 
        print ">" scaffold[1], "dna_sm:scaffold scaffold:GRCh38:" scaffold[1] ":1:1000000:1 ALT"; 
    } 
    next  
} 
{ print $0 }' data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.cdna.all.fa > data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.genome_like.fa


In [1]:
%%bash
TRANSCRIPT_FILE='data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.genome_like.fa'

echo "Transcript file: $TRANSCRIPT_FILE"
echo "Started..."
rm -rf data/Bowtie_index
mkdir data/Bowtie_index

bowtie2-build \
-f $TRANSCRIPT_FILE \
data/Bowtie_index/hg38_cdna_index \
--threads 8
echo "Done!"

# bowtie2-inspect -s data/Bowtie_index/hg38_cdna_index


/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory


Transcript file: data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.genome_like.fa
Started...
Settings:
  Output files: "data/Bowtie_index/hg38_cdna_index.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 32
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.genome_like.fa


Building a SMALL index


Reading reference sizes
  Time reading reference sizes: 00:00:03
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:02
bmax according to bmaxDivN setting: 12344445
Using parameters --bmax 9258334 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9258334 --dcv 1024
Constructing suffix-array element generator
Building DifferenceCoverSample
  Building sPrime
  Building sPrimeOrder
  V-Sorting samples
  V-Sorting samples time: 00:00:06
  Allocating rank array
  Ranking v-sort output
  Ranking v-sort output time: 00:00:03
  Invoking Larsson-Sadakane on ranks
  Invoking Larsson-Sadakane on ranks time: 00:00:03
  Sanity-checking and returning
Building samples
Reserving space for 86 sample suffixes
Generating random suffixes
QSorting 86 sample offsets, eliminating duplicates
QSorting sample offsets, eliminating duplicates time: 00:00:00
Multikey Q

Renaming data/Bowtie_index/hg38_cdna_index.3.bt2.tmp to data/Bowtie_index/hg38_cdna_index.3.bt2
Renaming data/Bowtie_index/hg38_cdna_index.4.bt2.tmp to data/Bowtie_index/hg38_cdna_index.4.bt2
Renaming data/Bowtie_index/hg38_cdna_index.1.bt2.tmp to data/Bowtie_index/hg38_cdna_index.1.bt2
Renaming data/Bowtie_index/hg38_cdna_index.2.bt2.tmp to data/Bowtie_index/hg38_cdna_index.2.bt2
Renaming data/Bowtie_index/hg38_cdna_index.rev.1.bt2.tmp to data/Bowtie_index/hg38_cdna_index.rev.1.bt2
Renaming data/Bowtie_index/hg38_cdna_index.rev.2.bt2.tmp to data/Bowtie_index/hg38_cdna_index.rev.2.bt2


Done!


# Run Bowtie2
Started ~14:00, ended ~14:50 with one core.

Started ~15:00, ended ~15:20 with two cores

In [ ]:
%%bash

mkdir -p data/alignment_Bowtie2_subset

# Loop over all FASTQ files ending in _1.fastq.gz in the subset_trimmed_RNA-seq folder
for R1 in data/subset_trimmed_RNA-seq/*_1.fastq.gz; do
  
  # Extract the base name 
  BN=$(basename "$R1")
  
  # Remove the _1.fastq.gz suffix to get the sample prefix 
  SAMPLE=${BN%%_1.fastq.gz}
  
  # Construct the path to the mate 2 file 
  R2="data/subset_trimmed_RNA-seq/${SAMPLE}_2.fastq.gz"
  
  echo "Aligning $SAMPLE with Bowtie2..."
  
  # Run Bowtie2 in local alignment mode for the paired-end reads
  bowtie2 --local \
    -x data/Bowtie_index/hg38_cdna_index \
    -1 "$R1" -2 "$R2" \
    -S data/alignment_Bowtie2_subset/${SAMPLE}.sam \
    --threads 4
  
  # Convert the SAM to a sorted BAM and create an index using samtools
  samtools view -bS data/alignment_Bowtie2_subset/${SAMPLE}.sam | samtools sort -o data/alignment_Bowtie2_subset/${SAMPLE}.bam
  samtools index data/alignment_Bowtie2_subset/${SAMPLE}.bam
  
  # Remove the SAM file to save disk space
  rm data/alignment_Bowtie2_subset/${SAMPLE}.sam
  
  echo "Done aligning $SAMPLE"
  echo "-----------------------------------------"
done


/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory


Aligning ERR950158_trimmed with Bowtie2...


499532 reads; of these:
  499532 (100.00%) were paired; of these:
    261310 (52.31%) aligned concordantly 0 times
    62637 (12.54%) aligned concordantly exactly 1 time
    175585 (35.15%) aligned concordantly >1 times
    ----
    261310 pairs aligned concordantly 0 times; of these:
      1885 (0.72%) aligned discordantly 1 time
    ----
    259425 pairs aligned 0 times concordantly or discordantly; of these:
      518850 mates make up the pairs; of these:
        486970 (93.86%) aligned 0 times
        6480 (1.25%) aligned exactly 1 time
        25400 (4.90%) aligned >1 times
51.26% overall alignment rate
[W::sam_hdr_create] Duplicated sequence "14" in file "data/alignment_Bowtie2_subset/ERR950158_trimmed.sam"
[W::sam_hdr_create] Duplicated sequence "14" in file "data/alignment_Bowtie2_subset/ERR950158_trimmed.sam"
[W::sam_hdr_create] Duplicated sequence "14" in file "data/alignment_Bowtie2_subset/ERR950158_trimmed.sam"
[W::sam_hdr_create] Duplicated sequence "14" in file "data/alig

In [ ]:
%%bash
samtools flagstat data/alignment_Bowtie2_subset/ERR950187_trimmed.bam

# Download .gtf file for summarization
https://ftp.ensembl.org/pub/release-113/gtf/homo_sapiens
/Homo_sapiens.GRCh38.113.gtf.gz

In [ ]:
%%script false --no-raise-error
%%bash
# -nc makes sure that if the file is already there, it won't be overwritten.
# -nc can be changed to -N, if it is of interest to import newer versions
# -P will save it to the directory specified, in this case data/cDNA_library
wget -nc https://ftp.ensembl.org/pub/release-113/gtf/homo_sapiens/Homo_sapiens.GRCh38.113.gtf.gz -P data/
gzip -d data/Homo_sapiens.GRCh38.113.gtf.gz
rm data/Homo_sapiens.GRCh38.113.gtf.gz

# Summarize with featureCounts

In [ ]:
%%bash
GTF_ANNOTATION='data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.113.gtf'

# path to the GTF annotation, output file, -p to specify paired-end reads 
# -C to remove pairs that have been mapped to different chromosomes 
# takes ~30 sek to run on two cores
# awk '$3 == "transcript"' 'data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.113.gtf' > data/homo_sapiens_ensembl/Homo_sapiens.GRCh38.113.transcripts.gtf

featureCounts \
-t exon \
-g gene_id \
-a $GTF_ANNOTATION \
-o data/Bowtie2_gene_counts/bowtie_subset_counts.txt \
-p \
-A data/featureCounts_alias/featureCounts_alias.txt \
--verbose \
data/alignment_Bowtie2_subset/*.bam
# --countReadPairs
# -A data/featureCounts_alias/featureCounts_alias.txt \ to include chromosome aliases
# --countReadPairs
# -A data/featureCounts_alias/featureCounts_alias.txt \ to include chromosome aliases

# Get gene counts with HTSeq
Because featureCounts is obviously not working.

In [ ]:
%%script false --no-raise-error
%%bash
htseq-count -f bam -t exon data/alignment_Bowtie2_subset/*.bam '$GTF_ANNOTATION'


# Remember to change access for everyone

In [ ]:
%%bash
chmod -R 777 . 

In [ ]:
! 